# Eventful Lists

keeping track of element changes in a list

In [1]:
from spectate import expose_as, watch

We use `watched_type` to create an `EventfulDict` that enables callbacks before and after an element of the dictionary is set. The funtion's signature requries the name of the new eventful type, a base class from which that new type should inherit, and the series of attributes on that base class which can register a callback. In this case we enable callbacks for methods which can set or delete items:

+ `__setitem__`
+ `__delitem__`
+ `pop`

In [2]:
EventfulList = expose_as('EventfulList', list, '__setitem__', '__delitem__', 'pop')

Beforebacks
-----------

+ Have a signature of ``(instance, call)``

    + ``instance`` is the owner of the method
    + ``call`` is a ``Bunch`` with the keys:

        + ``'name'`` - the name of the method which was called
        + ``'args'`` - the arguments which that method will call
        + ``'kwargs'`` - the keywords which that method will call

+ Can ``return`` a value which gets passed on to its respective afterback.
+ If an error is encountered, the wrapper will ``return`` the original ``call``
as if it were that of the beforeback, and set ``error`` in the ``answer``
bunch of its afterback. This way, if the base method call is valid, it's not
obstructed by a raised beforeback. Thus, beforeback errors should be handled
in an afterback.

Afterbacks
----------

+ Have a signature of ``(instance, answer)``

    + ``instance`` is the owner of the method
    + ``answer`` is a ``Bunch`` with the keys:

        + ``'name'`` - the name of the method which was called
        + ``'value'`` - the value returned by the method
        + ``'before'`` - the value returned by the respective beforeback

+ Should not ``return``

In [3]:
# beforeback

def pass_on_old_value(inst, call):
    index, old = call.args[0], inst[call.args[0]]
    return index, old

# afterback

def print_element_change(inst, answer):
    # answer.before = pass_on_old(call)
    index, old = answer.before
    try:
        new = inst[index]
    except:
        new = ''
    if new != old:
        print("{%s: %s} -> {%s: %s}" %
            (index, old, index, new))

Instances of `EventfulList` have a public attribute `instance_spectator` which is a `Spectator`. This object is relatively simple, and only has methods to add, delete, and trigger callbacks. To register a callback to a method, use the spectator's `callback` method. It's signatures is `(name, before, after)` where '`name`' is the name of the method you're observing, and `before` and `after` are a beforeback and an afterback. Callbacks are ultimately triggered by `MethodSpectator` descriptors that access the instance spectator (these descriptors are generally hidden from the user and shouldn't be tampered with).

In [4]:
elist = EventfulList([1, 2, 3])

methods = ('pop', '__delitem__', '__setitem__')

# callback accepts a method name string or,
# as in this case, a list or tuple of names
spectator = watch(elist)
spectator.callback(methods,
    before=pass_on_old_value,
    after=print_element_change)

In [5]:
# does not notify because
# values are the same
elist[0] = 1

In [6]:
# will notify when the
# values are different
elist[0] = 0

{0: 1} -> {0: 0}


In [7]:
# notify when deleting
# an element of the list
elist.pop(0)
del elist[0]

{0: 0} -> {0: 2}
{0: 2} -> {0: 3}


In [8]:
elist

[3]